### 1. Load the train and test datasets and create the corresponding dataloader

In [1]:
import torch
from torchvision import transforms

import hub

In [2]:
trainset = hub.load("hub://mohammadbashiri/npc-mouse1-train")
testset = hub.load("hub://mohammadbashiri/npc-mouse1-test")

hub://mohammadbashiri/npc-mouse1-train loaded successfully.


/usr/local/lib/python3.8/dist-packages/hub/core/dataset/dataset.py:548: UserWarning: Checking out dataset in read only mode as another machine has locked this version for writing.
  warnings.warn(


This dataset can be visualized at https://app.activeloop.ai/mohammadbashiri/npc-mouse1-train.
hub://mohammadbashiri/npc-mouse1-test loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/mohammadbashiri/npc-mouse1-test.


In [3]:
trainloader = trainset.pytorch(batch_size=16, shuffle=True, transform={'inputs': transforms.ToTensor(), 'targets': None, 'image_ids': None, 'trial_indices': None})
testloader = testset.pytorch(batch_size=16, shuffle=False, transform={'inputs': transforms.ToTensor(), 'image_ids': None, 'trial_indices': None})

In [4]:
#dataset_id = trainset.info.dataset_id
dataset_id = 'animal_0'

In [5]:
dataloaders = {"train": {dataset_id: trainloader},
              "validation": {dataset_id: trainloader},
              "test": {dataset_id: trainloader}}

In [ ]:
from nnfabrik.builder import get_model, get_trainer;

In [7]:
model_fn, model_config = ('cascade.models.stacked_core_full_gauss_readout',
 {'pad_input': False,
  'stack': -1,
  'layers': 1,
  'input_kern': 5,
  'hidden_dilation': 1,
  'hidden_kern': 3,
  'hidden_channels': 64,
  'depth_separable': True,
  'init_sigma': 0.1,
  'init_mu_range': 0.3,
  'gauss_type': 'full',
  'shifter': True,
  'shift_layers': 3})

In [8]:
model = get_model(model_fn, model_config, dataloaders, seed=1)

/src/neuralpredictors/neuralpredictors/layers/readouts/base.py:62: UserWarning: Use of 'gamma_readout' is deprecated. Please consider using the readout's feature-regularization parameter instead
  warnings.warn(
/src/neuralpredictors/neuralpredictors/layers/readouts/base.py:79: UserWarning: Readout is NOT initialized with mean activity but with 0!
  warnings.warn("Readout is NOT initialized with mean activity but with 0!")


In [9]:
trainer_fn, trainer_config = ('cascade.training.standard_trainer',
 {'max_iter': 2,
  'verbose': False,
  'lr_decay_steps': 4,
  'avg_loss': False,
  'lr_init': 0.009})

In [10]:
trainer = get_trainer(trainer_fn, trainer_config)

In [11]:
score, output, state_dict = trainer(model, dataloaders, seed=42)

Epoch 1: 100%|██████████| 13/13 [00:04<00:00,  3.13it/s]


[001|00/05] ---> 0.03848068905962665


Epoch 2: 100%|██████████| 13/13 [00:04<00:00,  3.11it/s]


[002|01/05] -/-> 0.03326271399990587
Restoring best model! 0.033263 ---> 0.038481
